# Configuration tutorial

This notebook shows the flexibility of the Config class and instantiate function in venturi, which allows instantiating any Python object you want inside a Python script from a yaml file or dictionary.

Usually, the Config class loads a configuration from a yaml file. But here we use a dictionary for better understanding of the class.

## Using as a simple namespace

A config object created from a dictionary or yaml file behaves like a Namespace object from argparse

In [1]:
from torch import nn

from venturi import Config, instantiate

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super().__init__()
        padding = kernel_size // 2
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.conv(x))
    
class SimpleCNN(nn.Module):
    """Simple CNN model."""
    def __init__(self, in_channels, out_channels, base_channels=16, kernel_size=3):
        super().__init__()
        self.layer1 = ConvBlock(in_channels, base_channels, kernel_size)
        self.layer2 = ConvBlock(base_channels, base_channels * 2, kernel_size)
        self.layer3 = nn.Conv2d(base_channels * 2, out_channels, kernel_size=1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

args_dict = {
    "in_channels": 3,
    "out_channels": 10,
    "base_channels": 16,
    "kernel_size": 3,
}
args = Config(args_dict)


model = SimpleCNN(
    in_channels=args.in_channels,
    out_channels=args.out_channels,
    base_channels=args.base_channels,
    kernel_size=args.kernel_size,
)

You can access elements with dot notation or use the object as a dictionary

In [2]:
print(args["kernel_size"])

3


## Instantiating objects

The special key *_target_* can be used to instantiate any class or function using the provided parameters at the same level of the key. 

In [3]:
args_dict = {
    "_target_": "SimpleCNN",
    "in_channels": 3,
    "out_channels": 10,
    "base_channels": 16,
    "kernel_size": 3,
}

cfg = Config(args_dict)
model = instantiate(cfg)
model

SimpleCNN(
  (layer1): ConvBlock(
    (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
  )
  (layer2): ConvBlock(
    (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
  )
  (layer3): Conv2d(32, 10, kernel_size=(1, 1), stride=(1, 1))
)

Note that this makes the dictionary define ALL information required to create the object. If you want to instantiate another class, **no code changes are required**. Just change the dictionary

In [4]:
args_dict = {
    "_target_": "torch.nn.Conv2d",
    "in_channels": 3,
    "out_channels": 10,
    "kernel_size": 3,
}

cfg = Config(args_dict)
model = instantiate(cfg)
model

Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))

You can even create complex nested objects

In [ ]:
args_dict = {
    "_target_": "torch.optim.lr_scheduler.PolynomialLR",
    "in_channels": 3,
    "out_channels": 10,
    "kernel_size": 3,
}

Other yaml files can be used for changing or adding new configurations

In [3]:
# config_override changes the experiment name and the optimizer
cfg.update_from_yaml("example_configs/config_override.yaml", allow_extra=True) # Allow new keys
print(cfg)

experiment_name: finetuning_experiment
seed: 42
model:
  _target_: Linear
  in_features: 128
  out_features: 10
  bias: true
optimizer:
  _target_: SGD
  lr: 0.01
  weight_decay: 0.0
  momentum: 0.9
data:
  batch_size: 32
  num_workers: 4
  path: ./data/dogs
  augmentations:
  - RandomCrop
  - RandomHorizontalFlip



The configuration can also be changed from a dictionary nested using the same structure as the yaml file

In [4]:
overrides = {
    "model": {
        "out_features": 2  # Change to 2 classes instead of 10
    },
    "data": {
        "batch_size": 64  # Change batch size
    }
}

cfg.update_from_dict(overrides) 
print(cfg)

experiment_name: finetuning_experiment
seed: 42
model:
  _target_: Linear
  in_features: 128
  out_features: 2
  bias: true
optimizer:
  _target_: SGD
  lr: 0.01
  weight_decay: 0.0
  momentum: 0.9
data:
  batch_size: 64
  num_workers: 4
  path: ./data/dogs
  augmentations:
  - RandomCrop
  - RandomHorizontalFlip



update_from_* can add new keys and update current ones. If you want to completely replace some configuration, just assing to it

In [5]:
cfg.data = Config("example_configs/config_data2.yaml").data
print(cfg)

experiment_name: finetuning_experiment
seed: 42
model:
  _target_: Linear
  in_features: 128
  out_features: 2
  bias: true
optimizer:
  _target_: SGD
  lr: 0.01
  weight_decay: 0.0
  momentum: 0.9
data:
  batch_size: 32
  path: ./data/cats



The final configuration can be saved to a yaml file

In [6]:
cfg.save("example_configs/config_final.yaml")

The class indicated in _target_ can be **instantiated** with all parameters on the same level (unless they begin with _)

In [7]:
model = instantiate(cfg.model)
model

Linear(in_features=128, out_features=2, bias=True)

Lazy instantiation is also supported in case the constructor needs additional objects created at runtime

In [8]:
optimizer_factory = instantiate(cfg.optimizer, partial=True)
optimizer = optimizer_factory(model.parameters())
optimizer


SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0
)